## Import package

In [1]:
import pandas as pd
import os
import numpy as np
from collections import defaultdict
from tqdm import tqdm

pd.options.mode.chained_assignment = None

In [2]:
prefix_path = '../data/mimiciii/1.4/'

## Split CHARTEVENTS to several dataset

In [7]:
count = 1
for reader in tqdm(pd.read_csv(prefix_path + 'CHARTEVENTS.csv', chunksize = 40000000)):
    reader.to_csv(prefix_path + 'CHARTEVENTS' + str(count) + '.csv')
    count += 1

0it [00:00, ?it/s]/home/toooot/anaconda3/envs/sepsis/lib/python3.8/site-packages/tqdm/std.py:1195: DtypeWarning: Columns (8,13,14) have mixed types. Specify dtype option on import or set low_memory=False.
  for obj in iterable:
4it [07:27, 110.95s/it]/home/toooot/anaconda3/envs/sepsis/lib/python3.8/site-packages/tqdm/std.py:1195: DtypeWarning: Columns (13) have mixed types. Specify dtype option on import or set low_memory=False.
  for obj in iterable:
6it [11:11, 111.30s/it]/home/toooot/anaconda3/envs/sepsis/lib/python3.8/site-packages/tqdm/std.py:1195: DtypeWarning: Columns (8,10,14) have mixed types. Specify dtype option on import or set low_memory=False.
  for obj in iterable:
9it [15:09, 101.08s/it]


## Load Data

In [3]:
paths = sorted(os.listdir(prefix_path))
remove_paths = []
for path in paths:
    if path[-4:] != '.csv':
        remove_paths.append(path)
for path in remove_paths:
    paths.remove(path)
paths.remove('CALLOUT.csv')
paths.remove('CAREGIVERS.csv')
paths.remove('CPTEVENTS.csv')
paths.remove('DATETIMEEVENTS.csv')
paths.remove('SERVICES.csv',)
paths.remove('TRANSFERS.csv')
paths.remove('PROCEDURES_ICD.csv')
paths.remove('PROCEDUREEVENTS_MV.csv')
paths.remove('MICROBIOLOGYEVENTS.csv')
paths.remove('D_CPT.csv')
paths

['ADMISSIONS.csv',
 'CHARTEVENTS1.csv',
 'CHARTEVENTS2.csv',
 'CHARTEVENTS3.csv',
 'CHARTEVENTS4.csv',
 'CHARTEVENTS5.csv',
 'CHARTEVENTS6.csv',
 'CHARTEVENTS7.csv',
 'CHARTEVENTS8.csv',
 'CHARTEVENTS9.csv',
 'DIAGNOSES_ICD.csv',
 'DRGCODES.csv',
 'D_ICD_DIAGNOSES.csv',
 'D_ICD_PROCEDURES.csv',
 'D_ITEMS.csv',
 'D_LABITEMS.csv',
 'ICUSTAYS.csv',
 'INPUTEVENTS_CV.csv',
 'INPUTEVENTS_MV.csv',
 'LABEVENTS.csv',
 'NOTEEVENTS.csv',
 'OUTPUTEVENTS.csv',
 'PATIENTS.csv',
 'PRESCRIPTIONS.csv']

In [4]:
datas = []
for path in tqdm(paths):
    datas.append(pd.read_csv(prefix_path + path))

  0%|                                                                                                                                                                               | 0/24 [00:00<?, ?it/s]/tmp/ipykernel_138439/136570088.py:3: DtypeWarning: Columns (10,15,16) have mixed types. Specify dtype option on import or set low_memory=False.
  datas.append(pd.read_csv(prefix_path + path))
 21%|██████████████████████████████████▊                                                                                                                                    | 5/24 [00:26<01:35,  5.04s/it]/tmp/ipykernel_138439/136570088.py:3: DtypeWarning: Columns (15) have mixed types. Specify dtype option on import or set low_memory=False.
  datas.append(pd.read_csv(prefix_path + path))
 29%|████████████████████████████████████████████████▋                                                                                                                      | 7/24 [00:37<01:29,  5.25s/it]/tmp/ipyker

## Clean Data to remain SEPSIS patient and sort Data

In [5]:
i = paths.index('D_ICD_DIAGNOSES.csv')
D_ICD_DIAGNOSES = datas[i].copy()
'''
D_ICD_DIAGNOSES.drop(D_ICD_DIAGNOSES[['sepsis' not in s.lower() for s in D_ICD_DIAGNOSES['LONG_TITLE']]].index, inplace=True)
D_ICD_DIAGNOSES.to_csv(prefix_path + paths[i])
'''
D_ICD_DIAGNOSES

,Unnamed: 0.2,Unnamed: 0.1,Unnamed: 0,ROW_ID,ICD9_CODE,SHORT_TITLE,LONG_TITLE
0,0,9049,9049,9050,77181,NB septicemia [sepsis],Septicemia [sepsis] of newborn
1,1,10304,10304,11403,99591,Sepsis,Sepsis
2,2,10305,10305,11404,99592,Severe sepsis,Severe sepsis
3,3,13293,13293,13564,67020,Puerperal sepsis-unsp,"Puerperal sepsis, unspecified as to episode of..."
4,4,13294,13294,13565,67022,Puerprl sepsis-del w p/p,"Puerperal sepsis, delivered, with mention of p..."
5,5,13295,13295,13566,67024,Puerperl sepsis-postpart,"Puerperal sepsis, postpartum condition or comp..."


In [6]:
i = paths.index('DIAGNOSES_ICD.csv')
datas[i].sort_values(by=['SUBJECT_ID', 'SEQ_NUM'], inplace=True)
DIAGNOSES_ICD = datas[i].copy()
'''
drop_index = []
for j in tqdm(DIAGNOSES_ICD.index):
    for code in D_ICD_DIAGNOSES['ICD9_CODE']:
        if str(DIAGNOSES_ICD.loc[j]['ICD9_CODE']) == code:
            break
    else:
        drop_index.append(j)
DIAGNOSES_ICD.drop(drop_index, inplace=True)
del drop_index
DIAGNOSES_ICD.to_csv(prefix_path + paths[i])
'''
DIAGNOSES_ICD

,Unnamed: 0,ROW_ID,SUBJECT_ID,HADM_ID,SEQ_NUM,ICD9_CODE
0,747,140,21,111970,11.0,99592
1,914,307,38,185910,3.0,99592
2,1084,477,61,189535,8.0,99591
3,1097,490,62,116009,4.0,99591
4,276,505,64,172056,3.0,99591
...,...,...,...,...,...,...
5404,648640,650341,99836,101018,8.0,99592
5405,649024,650470,99865,189346,4.0,99592
5406,649529,650720,99912,189380,8.0,99592
5407,648171,650990,99985,176670,7.0,99592


In [84]:
dataset = DIAGNOSES_ICD[['SUBJECT_ID', 'HADM_ID']].copy()
dataset.sort_values(by=['SUBJECT_ID', 'HADM_ID'], inplace=True)
dataset.index = range(len(dataset))
dataset

,SUBJECT_ID,HADM_ID
0,21,111970
1,38,185910
2,61,189535
3,62,116009
4,64,172056
...,...,...
5404,99836,101018
5405,99865,189346
5406,99912,189380
5407,99985,176670


In [ ]:
'''
dict1 = defaultdict(list)
for i in range(len(dataset)):
    dict1[dataset['SUBJECT_ID'][i]].append(dataset['HADM_ID'][i])

for i in tqdm(range(len(datas))):
    if i == paths.index('DIAGNOSES_ICD.csv'):
        continue
    if 'SUBJECT_ID' in datas[i].columns and 'HADM_ID' in datas[i].columns:
        drop_index = []
        for j in datas[i].index:
            if datas[i]['HADM_ID'][j] not in dict1[datas[i]['SUBJECT_ID'][j]]:
                drop_index.append(j)
        datas[i].drop(drop_index, inplace=True)
        del drop_index
        # datas[i].sort_values(by=['SUBJECT_ID', 'HADM_ID'], inplace=True)
    elif 'SUBJECT_ID' in datas[i].columns:
        datas[i] = datas[i][[id in dataset['SUBJECT_ID'].values for id in datas[i]['SUBJECT_ID']]]
        # datas[i].sort_values(by=['SUBJECT_ID'], inplace=True)

    datas[i].to_csv(prefix_path + paths[i])
    
del dict1

        
i = paths.index('D_ICD_PROCEDURES.csv')
datas[i].sort_values(by=['ICD9_CODE'], inplace=True)
datas[i].to_csv(prefix_path + paths[i])

i = paths.index('D_ITEMS.csv')
datas[i].sort_values(by=['ITEMID'], inplace=True)
datas[i].to_csv(prefix_path + paths[i])

i = paths.index('D_LABITEMS.csv')
datas[i].sort_values(by=['ITEMID'], inplace=True)
datas[i].to_csv(prefix_path + paths[i])
'''
for i in tqdm(range(len(datas))):
    for column in datas[i].columns:
        if 'TIME' in column or 'DATE' in column or 'DOB' == column or 'DOD' == column or 'DOD_HOSP' == column or 'DOD_SSN' == column:
            datas[i][column] = datas[i][column].apply(lambda x : pd.Timestamp(x))

 33%|███████████████████████████████████████████████████████▋                                                                                                               | 8/24 [02:39<06:15, 23.44s/it]

# Look Data characteristics

In [ ]:
i = paths.index('ICUSTAYS.csv')
ICUSTAYS = datas[i].copy()
ICUSTAYS.head()

In [ ]:
i = paths.index('PATIENTS.csv')
PATIENTS = datas[i].copy()
PATIENTS.head()

In [ ]:
i = paths.index('ADMISSIONS.csv')
ADMISSIONS = datas[i].copy()
ADMISSIONS.drop(ADMISSIONS[ADMISSIONS['HAS_CHARTEVENTS_DATA'] == 0].index, inplace=True)
ADMISSIONS.head()

In [ ]:
i = paths.index('INPUTEVENTS_CV.csv')
INPUTEVENTS_CV = datas[i].copy()
INPUTEVENTS_CV.head()

In [ ]:
i = paths.index('INPUTEVENTS_CV.csv')
INPUTEVENTS_MV = datas[i].copy()
INPUTEVENTS_MV.head()

In [ ]:
i = paths.index('OUTPUTEVENTS.csv')
OUTPUTEVENTS = datas[i].copy()
OUTPUTEVENTS.head()

In [ ]:
i = paths.index('D_ITEMS.csv')
D_ITEMS = datas[i].copy()
D_ITEMS.head()

In [ ]:
i = paths.index('D_LABITEMS.csv')
D_LABITEMS = datas[i].copy()
D_LABITEMS.head()

In [ ]:
i = paths.index('DRGCODES.csv')
DRGCODES = datas[i].copy()
DRGCODES.head()

In [ ]:
i = paths.index('LABEVENTS.csv')
LABEVENTS = datas[i].copy()
LABEVENTS.head()

In [ ]:
i = paths.index('PRESCRIPTIONS.csv')
PRESCRIPTIONS = datas[i].copy()
PRESCRIPTIONS.head()

In [ ]:
i = paths.index('CHARTEVENTS1.csv')
CHARTEVENTS1 = datas[i].copy()
CHARTEVENTS2 = datas[i + 1].copy()
CHARTEVENTS3 = datas[i + 2].copy()
CHARTEVENTS4 = datas[i + 3].copy()
CHARTEVENTS5 = datas[i + 4].copy()
CHARTEVENTS6 = datas[i + 5].copy()
CHARTEVENTS7 = datas[i + 6].copy()
CHARTEVENTS8 = datas[i + 7].copy()
CHARTEVENTS9 = datas[i + 8].copy()
CHARTEVENTS1.head()

# Features

In [85]:
# last row don't understand meaning
features = ['Shock_Index', 'elixhauser', 'SIRS', 'gender', 're_admission', 'GCS', 'SOFA', 'age', 'Albumin',
           'Arterial_pH', 'Calcium', 'Glucose', 'Hemoglobin', 'Magnesium', 'PTT', 'Potassium', 'SGPT', 
           'Arterial_blood_gas', 'BUN', 'HCO3', 'INR', 'Arterial_lactate', 'CO2', 'Creatinine', 
           'Ionised_Ca', 'PT', 'Platelets_count', 'SGOT', 'Total_bili', 'WBC_count', 'DiaBP', 'SysBP', 
           'MeanBP', 'PaCO2', 'PaO2', 'FiO2', 'PaO_FiO2_ratio', 'RR', 'Temp_C', 'Weight_kg', 'HR', 'SpO2'
           'input_total_tev', 'input_4hourly_tev', 'output_total', 'output_4hourly', 'bloc', 'mechvent']

binary_features = ['gender', 're_admission', 'mechvent']
norm_features = ['Shock_Index', 'elixhauser', 'SIRS', 'GCS', 'SOFA', 'age', 'Albumin', 'Arterial_pH', 
                'Calcium', 'Glucose', 'Hemoglobin', 'Magnesium', 'PTT', 'Potassium', 'Arterial_blood_gas', 
                'HCO3', 'Arterial_lactate', 'CO2', 'Ionised_Ca', 'PT', 'Platelets_count', 'WBC_count', 
                'DiaBP', 'SysBP', 'MeanBP', 'PaCO2', 'PaO2', 'FiO2', 'PaO_FiO2_ratio', 'RR', 'Temp_C', 
                'Weight_kg', 'HR', ]
log_features = ['SGPT', 'BUN', 'INR', 'Creatinine', 'SGOT', 'Total_bili', 'SpO2', 'input_total_tev',
                'input_4hourly_tev', 'output_total', 'output_4hourly', 'bloc']

## Add gender and age to dataset

In [86]:
dataset['DOB'] = pd.Series(['NaN'] * len(dataset))
dataset['Gender'] = pd.Series(['NaN'] * len(dataset))

for i in PATIENTS.index:
    subject_id = PATIENTS['SUBJECT_ID'].loc[i]
    gender = PATIENTS['GENDER'].loc[i]
    dob = PATIENTS['DOB'].loc[i]
    index = dataset[dataset['SUBJECT_ID'] == subject_id].index
    dataset['DOB'].loc[index] = dob
    dataset['Gender'].loc[index] = gender

dataset

,SUBJECT_ID,HADM_ID,DOB,Gender
0,21,111970,2047-04-04 00:00:00,M
1,38,185910,2090-08-31 00:00:00,M
2,61,189535,2063-10-21 00:00:00,M
3,62,116009,2044-05-08 00:00:00,M
4,64,172056,2116-06-27 00:00:00,F
...,...,...,...,...
5404,99836,101018,2064-04-13 00:00:00,F
5405,99865,189346,2107-06-25 00:00:00,F
5406,99912,189380,2068-09-26 00:00:00,M
5407,99985,176670,2127-04-08 00:00:00,M


## Add death time to dataset

In [87]:
dataset['DEATHTIME'] = pd.Series(['NaN'] * len(dataset))

for i in ADMISSIONS.index:
    subject_id = ADMISSIONS['SUBJECT_ID'].loc[i]
    hadm_id = ADMISSIONS['HADM_ID'].loc[i]
    death_time = ADMISSIONS['DEATHTIME'].loc[i]
    index = dataset[(dataset['SUBJECT_ID'] == subject_id) & (dataset['HADM_ID'] == hadm_id)].index
    dataset['DEATHTIME'].loc[index] = death_time   
    
dataset

,SUBJECT_ID,HADM_ID,DOB,Gender,DEATHTIME
0,21,111970,2047-04-04 00:00:00,M,2135-02-08 02:08:00
1,38,185910,2090-08-31 00:00:00,M,NaT
2,61,189535,2063-10-21 00:00:00,M,2119-02-03 01:35:00
3,62,116009,2044-05-08 00:00:00,M,NaT
4,64,172056,2116-06-27 00:00:00,F,NaT
...,...,...,...,...,...
5404,99836,101018,2064-04-13 00:00:00,F,NaT
5405,99865,189346,2107-06-25 00:00:00,F,2184-12-30 00:01:00
5406,99912,189380,2068-09-26 00:00:00,M,NaT
5407,99985,176670,2127-04-08 00:00:00,M,NaT


## Add ICUSTAY_ID to dataset

In [88]:
dataset['ICUSTAY_ID'] = pd.Series(['NaN'] * len(dataset))
dataset['Age'] = pd.Series(['NaN'] * len(dataset))
dataset['INTIME'] = pd.Series(['NaN'] * len(dataset))
dataset['OUTTIME'] = pd.Series(['NaN'] * len(dataset))

# use intime - dob to calculate age
for i in ICUSTAYS.index:
    subject_id = ICUSTAYS['SUBJECT_ID'].loc[i]
    hadm_id = ICUSTAYS['HADM_ID'].loc[i]
    icustay_id = ICUSTAYS['ICUSTAY_ID'].loc[i]
    intime = ICUSTAYS['INTIME'].loc[i]
    outtime = ICUSTAYS['OUTTIME'].loc[i]
    index = dataset[(dataset['SUBJECT_ID'] == subject_id) & (dataset['HADM_ID'] == hadm_id)].index
    if dataset['ICUSTAY_ID'].loc[index[0]] == 'NaN':
        dataset['ICUSTAY_ID'].loc[index[0]] = icustay_id
        dataset['INTIME'].loc[index[0]] = intime
        dataset['OUTTIME'].loc[index[0]] = outtime
        dataset['Age'].loc[index[0]] = (intime.to_pydatetime() - dataset['DOB'].loc[index[0]].to_pydatetime()).days / 365
    else:
        dataset.loc[len(dataset)] = [dataset['SUBJECT_ID'].loc[index[0]],
                                     dataset['HADM_ID'].loc[index[0]],
                                     dataset['DOB'].loc[index[0]],
                                     dataset['Gender'].loc[index[0]],
                                     dataset['DEATHTIME'].loc[index[0]],
                                     icustay_id,
                                     (intime.to_pydatetime() - dataset['DOB'].loc[index[0]].to_pydatetime()).days / 365,
                                     intime,
                                     outtime]

dataset.sort_values(by=['SUBJECT_ID', 'INTIME'])
dataset = dataset[['SUBJECT_ID', 'HADM_ID', 'ICUSTAY_ID', 'INTIME', 'OUTTIME', 'Gender', 'Age', 'DEATHTIME']]
dataset

/tmp/ipykernel_32246/459815132.py:30: FutureWarning: Inferring datetime64[ns] from data containing strings is deprecated and will be removed in a future version. To retain the old behavior explicitly pass Series(data, dtype=datetime64[ns])
  dataset.sort_values(by=['SUBJECT_ID', 'INTIME'])


,SUBJECT_ID,HADM_ID,ICUSTAY_ID,INTIME,OUTTIME,Gender,Age,DEATHTIME
0,21,111970,216859,2135-01-30 20:53:34,2135-02-08 05:38:46,M,87.882192,2135-02-08 02:08:00
1,38,185910,248910,2166-08-10 00:29:36,2166-09-04 13:39:23,M,75.991781,NaT
2,61,189535,217135,2119-01-20 15:58:00,2119-01-22 16:11:10,M,55.284932,2119-02-03 01:35:00
3,62,116009,216609,2113-02-15 00:20:44,2113-02-17 20:09:46,M,68.819178,NaT
4,64,172056,232593,2143-03-03 09:26:21,2143-03-05 17:14:25,F,26.69863,NaT
...,...,...,...,...,...,...,...,...
6209,96240,168044,201263,2105-05-28 15:44:21,2105-05-31 18:22:17,M,53.331507,NaT
6210,99096,190010,276912,2110-07-13 04:30:18,2110-07-18 00:07:21,M,56.750685,2110-07-21 14:10:00
6211,94767,184200,278639,2123-09-12 23:46:10,2123-09-14 20:29:08,M,80.715068,NaT
6212,97529,173784,278787,2141-02-09 09:19:59,2141-02-14 07:04:27,M,78.471233,2141-02-13 21:50:00


## Add re-admission to dataset

In [89]:
dataset['re_admission'] = pd.Series(['NaN'] * len(dataset))

dataset['re_admission'].loc[0] = 0
for i in dataset.index[1:]:
    if dataset['SUBJECT_ID'].loc[i] == dataset['SUBJECT_ID'].loc[i - 1]:
        dataset['re_admission'].loc[i] = 1
    else:
        dataset['re_admission'].loc[i] = 0

dataset

,SUBJECT_ID,HADM_ID,ICUSTAY_ID,INTIME,OUTTIME,Gender,Age,DEATHTIME,re_admission
0,21,111970,216859,2135-01-30 20:53:34,2135-02-08 05:38:46,M,87.882192,2135-02-08 02:08:00,0
1,38,185910,248910,2166-08-10 00:29:36,2166-09-04 13:39:23,M,75.991781,NaT,0
2,61,189535,217135,2119-01-20 15:58:00,2119-01-22 16:11:10,M,55.284932,2119-02-03 01:35:00,0
3,62,116009,216609,2113-02-15 00:20:44,2113-02-17 20:09:46,M,68.819178,NaT,0
4,64,172056,232593,2143-03-03 09:26:21,2143-03-05 17:14:25,F,26.69863,NaT,0
...,...,...,...,...,...,...,...,...,...
6209,96240,168044,201263,2105-05-28 15:44:21,2105-05-31 18:22:17,M,53.331507,NaT,0
6210,99096,190010,276912,2110-07-13 04:30:18,2110-07-18 00:07:21,M,56.750685,2110-07-21 14:10:00,0
6211,94767,184200,278639,2123-09-12 23:46:10,2123-09-14 20:29:08,M,80.715068,NaT,0
6212,97529,173784,278787,2141-02-09 09:19:59,2141-02-14 07:04:27,M,78.471233,2141-02-13 21:50:00,0


# Split time

In [94]:
hour_period = '10:00:00'

In [97]:
for hour in range(7, 10):
    hour_period = f'0{hour}:00:00'
    
    index = 0
    split = pd.DataFrame(columns = dataset.columns)
    for i in tqdm(dataset.index):
        intime, outtime = dataset['INTIME'].loc[i], dataset['OUTTIME'].loc[i]
        if intime == 'NaN' or 'outtime' == 'NaN':
            continue
        while outtime - intime > pd.Timedelta(hour_period):
            #['SUBJECT_ID', 'HADM_ID', 'ICUSTAY_ID', 'INTIME', 'OUTTIME', 'Gender', 'Age', 'DEATHTIME', 're_admission']
            split.loc[index] = [
                dataset['SUBJECT_ID'].loc[i],
                dataset['HADM_ID'].loc[i],
                dataset['ICUSTAY_ID'].loc[i],
                intime,
                intime + pd.Timedelta(hour_period),
                dataset['Gender'].loc[i],
                dataset['Age'].loc[i],
                dataset['DEATHTIME'].loc[i],
                dataset['re_admission'].loc[i],
            ]
            intime += pd.Timedelta(hour_period)
            index += 1

        split.loc[index] = [
            dataset['SUBJECT_ID'].loc[i],
            dataset['HADM_ID'].loc[i],
            dataset['ICUSTAY_ID'].loc[i],
            intime,
            outtime,
            dataset['Gender'].loc[i],
            dataset['Age'].loc[i],
            dataset['DEATHTIME'].loc[i],
            dataset['re_admission'].loc[i],
        ]
        
    split.to_csv(f'../data/mimiciii/dataset_split_{hour}_hour.csv')

100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 6214/6214 [07:00<00:00, 14.77it/s]


In [96]:
split.to_csv('../data/mimiciii/dataset_split_10_hour.csv')

# Features

In [ ]:
# last row don't understand meaning
features = ['Shock_Index', 'elixhauser', 'SIRS', 'gender', 're_admission', 'GCS', 'SOFA', 'age', 'Albumin',
           'Arterial_pH', 'Calcium', 'Glucose', 'Hemoglobin', 'Magnesium', 'PTT', 'Potassium', 'SGPT', 
           'Arterial_blood_gas', 'BUN', 'HCO3', 'INR', 'Arterial_lactate', 'CO2', 'Creatinine', 
           'Ionised_Ca', 'PT', 'Platelets_count', 'SGOT', 'Total_bili', 'WBC_count', 'DiaBP', 'SysBP', 
           'MeanBP', 'PaCO2', 'PaO2', 'FiO2', 'PaO_FiO2_ratio', 'RR', 'Temp_C', 'Weight_kg', 'HR', 'SpO2'
           'input_total_tev', 'input_4hourly_tev', 'output_total', 'output_4hourly', 'bloc', 'mechvent']

binary_features = ['gender', 're_admission', 'mechvent']
norm_features = ['Shock_Index', 'elixhauser', 'SIRS', 'GCS', 'SOFA', 'age', 'Albumin', 'Arterial_pH', 
                'Calcium', 'Glucose', 'Hemoglobin', 'Magnesium', 'PTT', 'Potassium', 'Arterial_blood_gas', 
                'HCO3', 'Arterial_lactate', 'CO2', 'Ionised_Ca', 'PT', 'Platelets_count', 'WBC_count', 
                'DiaBP', 'SysBP', 'MeanBP', 'PaCO2', 'PaO2', 'FiO2', 'PaO_FiO2_ratio', 'RR', 'Temp_C', 
                'Weight_kg', 'HR', ]
log_features = ['SGPT', 'BUN', 'INR', 'Creatinine', 'SGOT', 'Total_bili', 'SpO2', 'input_total_tev',
                'input_4hourly_tev', 'output_total', 'output_4hourly', 'bloc']

## Load variable range

In [ ]:
var_range = pd.read_csv('../data/mimiciii/variable_range.csv')
var_range.index = var_range.iloc[:, 0]
var_range = var_range.iloc[:, 1:]
var_range['GROUP_ID'] = var_range['GROUP_ID'].apply(lambda x : [int(s) for s in x.split(',')] if type(x) == type('') else x)
var_range

## Merge features to dataset

In [ ]:
def merge(dataset, feature, EVENTS, var_range, have_icustay_id=True):
    # find all feature records in EVENTS
    event = EVENTS[[id in var_range.loc[feature]['GROUP_ID'] for id in EVENTS['ITEMID']]]

    for i in range(len(event)):
        subject_id = event.iloc[i]['SUBJECT_ID']
        hadm_id = event.iloc[i]['HADM_ID']
        if have_icustay_id:
            icustay_id = event.iloc[i]['ICUSTAY_ID']
        time = event.iloc[i]['CHARTTIME']
        
        # detect outlier value, if out of outlier range then set to missing value
        value = event.iloc[i]['VALUENUM']
        if var_range.loc[feature].notnull()[0]:
            if value > var_range.loc[feature]['OUTLIER HIGH']:
                value = np.nan
            elif value > var_range.loc[feature]['VALID HIGH']:
                value = var_range.loc[feature]['VALID HIGH']
            elif value < var_range.loc[feature]['OUTLIER LOW']:
                value = np.nan
            elif value < var_range.loc[feature]['VALID LOW']:
                value = var_range.loc[feature]['VALID LOW']
        
        if have_icustay_id:
            index = (dataset[dataset['SUBJECT_ID'] == subject_id][dataset['HADM_ID'] == hadm_id][dataset['ICUSTAY_ID'] == icustay_id]).index
        else:
            index = (dataset[dataset['SUBJECT_ID'] == subject_id][dataset['HADM_ID'] == hadm_id]).index
        
        for idx in index:
            if time - dataset.iloc[idx]['STARTTIME'] >= pd.Timedelta('0') and time - dataset.iloc[idx]['ENDTIME'] <= pd.Timedelta('0'):
                if dataset['records'][idx] == 0:
                    dataset[feature][idx] = value
                else:
                    record = dataset['records'][idx]
                    # use mean when multiple records in same period
                    dataset[feature][idx] = value / (record + 1) + record / (record + 1) * dataset[feature][idx]
                dataset['records'][idx] += 1
                
                if have_icustay_id:
                    break
                    

path = f'../data/mimiciii/dataset_split_2_hour.csv'

dataset = pd.read_csv(path)

# change time column data's type to timestamp type
dataset['INTIME'] = dataset['INTIME'].apply(lambda x : pd.Timestamp(x))
dataset['OUTTIME'] = dataset['OUTTIME'].apply(lambda x : pd.Timestamp(x))
dataset['DEATHTIME'] = dataset['DEATHTIME'].apply(lambda x : pd.Timestamp(x))
dataset.columns = ['Unnamed:0', 'SUBJECT_ID', 'HADM_ID', 'ICUSTAY_ID', 'STARTTIME', 'ENDTIME', 'Gender',
                   'Age', 'DEATHTIME', 're_admission']
dataset = dataset.iloc[:, 1:]
# dataset.head()                    


for feature in tqdm(var_range.index):
    # count records in same period
    dataset['records'] = pd.Series([0] * len(dataset), dtype = np.int64)

    # add new feature column
    dataset[feature] = pd.Series([np.nan] * len(dataset), dtype = np.float64)

    # add data to new feature column with corresponding subject_id, hadm_id, icustay_id and time
    merge(dataset, feature, LABEVENTS, var_range, have_icustay_id = False)
    merge(dataset, feature, OUTPUTEVENTS, var_range, have_icustay_id = True)
    merge(dataset, feature, CHARTEVENTS1, var_range, have_icustay_id = True)
    merge(dataset, feature, CHARTEVENTS2, var_range, have_icustay_id = True)
    merge(dataset, feature, CHARTEVENTS3, var_range, have_icustay_id = True)
    merge(dataset, feature, CHARTEVENTS4, var_range, have_icustay_id = True)
    merge(dataset, feature, CHARTEVENTS5, var_range, have_icustay_id = True)
    merge(dataset, feature, CHARTEVENTS6, var_range, have_icustay_id = True)
    merge(dataset, feature, CHARTEVENTS7, var_range, have_icustay_id = True)
    merge(dataset, feature, CHARTEVENTS8, var_range, have_icustay_id = True)
    merge(dataset, feature, CHARTEVENTS9, var_range, have_icustay_id = True)

dataset.to_csv(f'../data/mimiciii/dataset_split_2_hour_after_merge.csv')